# Metrics for RPN and GCViT


In [1]:
from torch import Tensor
import torch
import pandas as pd
from project.dataset import Dataset, VALDODataset
from project.preprocessing import NiftiToTensorTransform
from torch.utils.data import DataLoader
from project.utils import collatev2, compute_statistics

d:\Programming Files\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.21 (you have 1.4.7). Upgrade using: pip install --upgrade albumentations


In [2]:
ds = Dataset()

data = pd.read_csv('targets.csv')
data.shape

(7986, 7)

In [3]:
data = data.query('has_microbleed_slice == 1').reset_index(drop=True)

In [4]:
from sklearn.model_selection import train_test_split

def make_loaders(data,
                 cohort,
                 batch_size,
                 test_size=0.2,
                 random_state=12,
                 target_shape=(300, 300),
                 rpn_mode=True,
                 logger=None
                ):
    data = data[data.cohort == cohort]
    
    s = f'Creating loaders for Cohort {cohort}\n'

    data_train, data_test = train_test_split(
        data,
        test_size=test_size,
        random_state=random_state
    )

    s += f'TRAIN & TEST: {data_train.shape, data_test.shape}\n'

    paths = data_train.mri.unique().tolist()
    s += f'Total Unique MRI Samples in data_train: {len(paths)}\n'
    
    global_min, global_max = compute_statistics(paths)
    s += f'GLOBAL MIN & MAX {global_min, global_max}\n'

    transform = NiftiToTensorTransform(
        target_shape=target_shape,
        rpn_mode=rpn_mode,
        normalization=(global_min, global_max)
    )

    train_set = VALDODataset(
        cases=data_train.mri.tolist(),
        masks=data_train.masks.tolist(),
        target=data_train.target.tolist(),
        transform=transform
    )
    val_set = VALDODataset(
        cases=data_test.mri.tolist(),
        masks=data_test.masks.tolist(),
        target=data_test.target.tolist(),
        transform=transform
    )

    if logger != None:
        logger.info(s)
    else:
        print(s)
    
    return train_set, val_set

In [5]:
t1, v1 = make_loaders(
    data=data,
    cohort=1,
    batch_size=4
)

pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1

Creating loaders for Cohort 1
TRAIN & TEST: ((36, 7), (9, 7))
Total Unique MRI Samples in data_train: 8
GLOBAL MIN & MAX (0.0, 1417.92822265625)



In [6]:
t3, v3 = make_loaders(
    data=data,
    cohort=3,
    batch_size=4
)

Creating loaders for Cohort 3
TRAIN & TEST: ((37, 7), (10, 7))
Total Unique MRI Samples in data_train: 26
GLOBAL MIN & MAX (0.0, 664.0)



In [7]:
from torch.utils.data import ConcatDataset

train_set = ConcatDataset([t1, t3])
val_set = ConcatDataset([v1, v3])

In [8]:
tl = DataLoader(
    train_set,
    shuffle=True,
    batch_size=20,
    collate_fn=collatev2
)
vl = DataLoader(
    val_set,
    shuffle=True,
    batch_size=20,
    collate_fn=collatev2
)


## RPN metrics


In [9]:
from project.model import RPN

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

rpn_model = RPN(
    input_dim=512,
    output_dim=4,
    image_size=300,
    nh=4,
    pretrained=True
).to(device)

d:\Programming Files\Python\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [10]:
rpn_model.load_state_dict(torch.load('weights/RPN_cohorts_1_3_50_epochs_mse_loss_Nov_06_2024_205918.pt'))

<All keys matched successfully>

### IOU


In [11]:
from torchmetrics.functional.detection import intersection_over_union
from torchvision.ops import box_area
from typing import Tuple


In [12]:
def _upcast(t: Tensor) -> Tensor:
    # Protects from numerical overflows in multiplications by upcasting to the equivalent higher type
    if t.is_floating_point():
        return t if t.dtype in (torch.float32, torch.float64) else t.float()
    else:
        return t if t.dtype in (torch.int32, torch.int64) else t.int()

def _box_inter_union(boxes1: Tensor, boxes2: Tensor) -> Tuple[Tensor, Tensor]:
    area1 = box_area(boxes1)
    area2 = box_area(boxes2)

    lt = torch.max(boxes1[:, None, :2], boxes2[:, :2])  # [N,M,2]
    rb = torch.min(boxes1[:, None, 2:], boxes2[:, 2:])  # [N,M,2]

    wh = _upcast(rb - lt).clamp(min=0)  # [N,M,2]
    inter = wh[:, :, 0] * wh[:, :, 1]  # [N,M]

    union = area1[:, None] + area2 - inter

    return inter, union

In [13]:
image_size = 300

## Train Data Loader


In [14]:
tl_iou_scores = []
tl_preds = []
tl_truth = []
tl_cases = []
tl_targets = []
tl_precision_scores = []
tl_recall_scores = []
tl_f1_scores = []
tl_counter = 0

with torch.no_grad():
    for batch in tl:
        for i in range(len(batch)):
            slices, masks, target, case = batch[i]
            x = slices.squeeze(1).repeat(1, 3, 1, 1).float().to(device)
            T = masks.squeeze(1).float().to(device)/image_size

            y = rpn_model(x, target)

            tl_preds.append((y *  image_size).squeeze().detach().cpu().numpy())
            tl_truth.append((T[target] * image_size).squeeze().detach().cpu().numpy())
            tl_cases.append(case)
            tl_targets.append(target)

            tl_iou_score = intersection_over_union(y * image_size, T[target] * image_size)

            tl_inter, tl_union = _box_inter_union(y * image_size, T[target] * image_size)

            tl_precision_score = (tl_inter / box_area(y * image_size))
            tl_recall_score = (tl_inter / box_area(T[target] * image_size))

            if any([tl_precision_score, tl_recall_score]) == 0:
                tl_f1_score = 0
                tl_f1_scores.append(tl_f1_score)
            else:
                tl_f1_score = (2 * (tl_precision_score * tl_recall_score)) / (tl_precision_score + tl_recall_score)
                tl_f1_scores.append(tl_f1_score.detach().cpu().numpy())

            tl_iou_scores.append(tl_iou_score.detach().cpu().numpy())
            tl_precision_scores.append(tl_precision_score.squeeze().squeeze().detach().cpu().numpy())
            tl_recall_scores.append(tl_recall_score.squeeze().squeeze().detach().cpu().numpy())
            tl_counter += 1


pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1

In [15]:
tl_score_dict = {
    'Case': tl_cases,
    'Target': tl_targets,
    'Ground_Truth': tl_truth,
    'Predictions': tl_preds,
    'IOU': tl_iou_scores,
    'Precision': tl_precision_scores,
    'Recall': tl_recall_scores,
    'F1': tl_f1_scores
}

tl_score_df = pd.DataFrame(tl_score_dict)

In [16]:
tl_score_df

,Case,Target,Ground_Truth,Predictions,IOU,Precision,Recall,F1
0,d:\Github\Thesis and ML Project\Dataset\VALDO_...,19,"[134.17969, 73.828125, 240.82031, 156.44531]","[157.22818, 87.86252, 237.61316, 138.9068]",0.4657243,1.0,0.4657243,[[0.6354869]]
1,d:\Github\Thesis and ML Project\Dataset\VALDO_...,35,"[92.578125, 111.328125, 212.10938, 230.85938]","[109.46289, 121.4304, 155.45508, 169.5667]",0.15495081,1.0,0.15495081,[[0.26832452]]
2,d:\Github\Thesis and ML Project\Dataset\VALDO_...,17,"[94.921875, 104.88281, 172.85156, 177.53906]","[94.61526, 139.07092, 122.45817, 157.87566]",0.0913597,0.98898757,0.09145273,[[0.16742362]]
3,d:\Github\Thesis and ML Project\Dataset\VALDO_...,19,"[20.507812, 86.13281, 91.40625, 167.57812]","[30.670006, 103.67617, 70.89777, 155.26727]",0.3594165,1.0,0.3594165,[[0.5287806]]
4,d:\Github\Thesis and ML Project\Dataset\VALDO_...,10,"[18.75, 62.109375, 140.625, 182.8125]","[41.99595, 86.165535, 114.54622, 145.35579]",0.2919148,1.0,0.2919148,[[0.4519103]]
...,...,...,...,...,...,...,...,...
68,d:\Github\Thesis and ML Project\Dataset\VALDO_...,13,"[1.171875, 134.76562, 121.875, 255.46875]","[31.216421, 161.09384, 110.042755, 243.97403]",0.44842005,1.0,0.44842005,[[0.6191851]]
69,d:\Github\Thesis and ML Project\Dataset\VALDO_...,18,"[111.91406, 151.75781, 172.85156, 212.69531]","[132.14326, 149.66022, 139.83173, 204.15344]",0.10801477,0.9615072,0.10848388,[[0.1949699]]
70,d:\Github\Thesis and ML Project\Dataset\VALDO_...,16,"[122.46094, 97.265625, 191.01562, 208.59375]","[123.80505, 104.928024, 154.35136, 184.90591]",0.3201007,1.0,0.32010072,[[0.48496407]]
71,d:\Github\Thesis and ML Project\Dataset\VALDO_...,32,"[71.484375, 100.19531, 134.17969, 162.30469]","[84.65934, 98.04216, 102.48688, 140.64664]",0.18338834,0.9494619,0.18519612,[[0.30993772]]


## Validation Data Loader


In [17]:
vl_iou_scores = []
vl_preds = []
vl_truth = []
vl_cases = []
vl_targets = []
vl_precision_scores = []
vl_recall_scores = []
vl_f1_scores = []
vl_counter = 0

with torch.no_grad():
    for batch in vl:
        for i in range(len(batch)):
            slices, masks, target, case = batch[i]
            x = slices.squeeze(1).repeat(1, 3, 1, 1).float().to(device)
            T = masks.squeeze(1).float().to(device)/image_size

            y = rpn_model(x, target)

            vl_preds.append((y *  image_size).squeeze().detach().cpu().numpy())
            vl_truth.append((T[target] * image_size).squeeze().detach().cpu().numpy())
            vl_cases.append(case)
            vl_targets.append(target)

            vl_iou_score = intersection_over_union(y * image_size, T[target] * image_size)

            vl_inter, vl_union = _box_inter_union(y * image_size, T[target] * image_size)

            vl_precision_score = (vl_inter / box_area(y * image_size))
            vl_recall_score = (vl_inter / box_area(T[target] * image_size))

            if any([vl_precision_score, vl_recall_score]) == 0:
                vl_f1_score = 0
                vl_f1_scores.append(vl_f1_score)
            else:
                vl_f1_score = (2 * (vl_precision_score * vl_recall_score)) / (vl_precision_score + vl_recall_score)
                vl_f1_scores.append(vl_f1_score.detach().cpu().numpy())

            vl_iou_scores.append(vl_iou_score.detach().cpu().numpy())
            vl_precision_scores.append(vl_precision_score.squeeze().squeeze().detach().cpu().numpy())
            vl_recall_scores.append(vl_recall_score.squeeze().squeeze().detach().cpu().numpy())
            vl_counter += 1


pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1

In [18]:
vl_score_dict = {
    'Case': vl_cases,
    'Target': vl_targets,
    'Ground_Truth': vl_truth,
    'Predictions': vl_preds,
    'IOU': vl_iou_scores,
    'Precision': vl_precision_scores,
    'Recall': vl_recall_scores,
    'F1': vl_f1_scores
}

vl_score_df = pd.DataFrame(vl_score_dict)

In [19]:
vl_score_df

,Case,Target,Ground_Truth,Predictions,IOU,Precision,Recall,F1
0,d:\Github\Thesis and ML Project\Dataset\VALDO_...,15,"[80.859375, 116.015625, 201.5625, 235.54688]","[112.96271, 112.21169, 145.991, 172.40048]",0.12796262,0.9368,0.12907693,[[0.2268916]]
1,d:\Github\Thesis and ML Project\Dataset\VALDO_...,14,"[2.34375, 135.9375, 121.875, 255.46875]","[68.025665, 149.52098, 130.67418, 203.80186]",0.1979628,0.8595469,0.20458055,[[0.33049908]]
2,d:\Github\Thesis and ML Project\Dataset\VALDO_...,18,"[45.703125, 147.65625, 165.23438, 267.1875]","[68.13995, 117.33127, 170.99318, 202.72847]",0.30169472,0.6087869,0.3742518,[[0.46354145]]
3,d:\Github\Thesis and ML Project\Dataset\VALDO_...,24,"[21.09375, 59.765625, 102.53906, 176.36719]","[61.147186, 99.77289, 121.926544, 172.24318]",0.27515808,0.68101865,0.3158673,[[0.43156695]]
4,d:\Github\Thesis and ML Project\Dataset\VALDO_...,26,"[110.15625, 119.53125, 233.20312, 241.40625]","[103.32042, 104.172714, 169.84447, 172.16055]",0.19180517,0.6945545,0.20947443,[[0.32187334]]
5,d:\Github\Thesis and ML Project\Dataset\VALDO_...,12,"[5.859375, 116.015625, 126.5625, 236.71875]","[52.03413, 132.91034, 89.099205, 187.7659]",0.139556,1.0,0.139556,[[0.24493048]]
6,d:\Github\Thesis and ML Project\Dataset\VALDO_...,23,"[106.640625, 76.171875, 226.17188, 196.875]","[130.23915, 73.80242, 213.99149, 169.5902]",0.5349291,0.97526354,0.54228675,[[0.6970082]]
7,d:\Github\Thesis and ML Project\Dataset\VALDO_...,11,"[20.507812, 151.17188, 81.44531, 212.10938]","[92.7193, 145.53961, 143.10303, 195.79063]",0.0,0.0,0.0,0
8,d:\Github\Thesis and ML Project\Dataset\VALDO_...,14,"[39.84375, 82.03125, 100.78125, 142.96875]","[50.429844, 90.35767, 106.030174, 161.87347]",0.52553684,0.6662073,0.7133777,[[0.6889861]]
9,d:\Github\Thesis and ML Project\Dataset\VALDO_...,22,"[16.40625, 69.140625, 135.35156, 162.89062]","[81.876114, 107.58381, 153.32457, 165.63576]",0.23964733,0.7130557,0.26522502,[[0.38663793]]


### Metrics


In [20]:
print('Metrics for Training Set')
print(f'IOU Score: {sum(tl_iou_scores) / len(tl_iou_scores)}')
print(f'Precision Score: {sum(tl_precision_scores) / len(tl_precision_scores)}')
print(f'Recall Score: {sum(tl_recall_scores) / len(tl_recall_scores)}')
print(f'F1 Score: {sum(tl_f1_scores) / len(tl_f1_scores)}')

Metrics for Training Set
IOU Score: 0.3194236594111952
Precision Score: 0.9591874096491565
Recall Score: 0.3223932170092243
F1 Score: [[0.46264282]]


In [21]:
print('Metrics for Validation Set')
print(f'IOU Score: {sum(vl_iou_scores) / len(vl_iou_scores)}')
print(f'Precision Score: {sum(vl_precision_scores) / len(vl_precision_scores)}')
print(f'Recall Score: {sum(vl_recall_scores) / len(vl_recall_scores)}')
print(f'F1 Score: {sum(vl_f1_scores) / len(vl_f1_scores)}')

Metrics for Validation Set
IOU Score: 0.23185888952330538
Precision Score: 0.608519487475094
Recall Score: 0.29430026600235387
F1 Score: [[0.35500062]]


#### Plotting of highest score


In [22]:
# import matplotlib.patches as patches
# import matplotlib.pyplot as plt

# num = iou_scores.index(max(iou_scores))

# slices, masks, target, case = dataset[num]

# truth_bbox = truth[num]
# predicted_bbox = preds[num]

# fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# ax.imshow(slices[target][0, 0, :], cmap='gray')
# # Create the bounding box rectangle
# truth_rect = patches.Rectangle(
#     (truth_bbox[0], truth_bbox[1]),  # (x_min, y_min)
#     truth_bbox[2] - truth_bbox[0],   # width
#     truth_bbox[3] - truth_bbox[1],   # height
#     linewidth=1, edgecolor='g', facecolor='none',  # red bounding box
#     label='Truth'
# )

# # Create the bounding box rectangle
# predicted_rect = patches.Rectangle(
#     (predicted_bbox[0], predicted_bbox[1]),  # (x_min, y_min)
#     predicted_bbox[2] - predicted_bbox[0],   # width
#     predicted_bbox[3] - predicted_bbox[1],   # height
#     linewidth=1, edgecolor='r', facecolor='none',  # red bounding box
#     label='Prediction'
# )

# ax.set_title(f'IOU:{iou_scores[num]}, Precision: {precision_scores[num]}, Recall: {recall_scores[num]}, F1: {f1_scores[num]}')

# # Add the rectangle to the axis
# ax.add_patch(truth_rect)
# ax.add_patch(predicted_rect)

# ax.legend()